In [78]:
from cmtj import Junction, Layer, CVector, Axis, ScalarDriver, AxialDriver, NullDriver
from cmtj.utils.linear import FieldScan
from tqdm.notebook import tqdm
from scipy.fft import fft, fftfreq
import numpy as np 
import pandas as pd 
import time 

In [140]:
demagTensor = [
    CVector(0., 0., 0.),
    CVector(0., 0., 0.),
    CVector(0., 0., 1.0)
]
dipoleTensor = [
    CVector(5.57049776248663e-4, 0., 0.),
    CVector(0., 0.00125355500286346, 0.),
    CVector(0., 0.0, -0.00181060482770131)
]

damping = 0.03
currentDensity = 1e11
beta = 1
spinPolarisation = 0.9


# current driver
mag_free = CVector(1, 0, 0)
mag_bottom = CVector(1, 0, 0)

l1 = Layer.createSTTLayer(id="free", 
           mag=mag_free, 
           anis=CVector(0, 0., 1.), 
           Ms=1.,
           thickness=1.e-9, 
           cellSurface=7e-10 * 7e-10, 
           demagTensor=demagTensor, 
           dipoleTensor=dipoleTensor,
           damping=damping,
           SlonczewskiSpacerLayerParameter=1.0,
           spinPolarisation=spinPolarisation,
           beta=beta)
l2 = Layer(id="bottom", 
           mag=mag_bottom, 
           anis=CVector(1., 0., 0.), 
           Ms=1.2, 
           thickness=1e-9, 
           cellSurface=7e-10 * 7e-10, 
           demagTensor=demagTensor, 
           dipoleTensor=dipoleTensor)

l1.setReferenceLayer(CVector(0, 0., 1.))
junction = Junction([l1, l2], "", 100, 200)

junction.setLayerAnisotropyDriver("free", ScalarDriver.getConstantDriver(550e3))
junction.setLayerAnisotropyDriver("bottom", ScalarDriver.getConstantDriver(700e3))
junction.setIECDriver("free", "bottom", ScalarDriver.getConstantDriver(-4.5e-5))


Hscan, Hvecs = FieldScan.amplitude_scan(-800e3, 800e3, 50, 5, 0)

int_step = 1e-12 
s_time = 4e-9
steps = int(s_time / int_step)
frequencies = np.arange(0, 48)*1e9
D = 25
spectrogram = np.zeros((len(Hscan), steps//D))
for hi, H in enumerate(tqdm(Hvecs, total=len(Hscan))):
    junction.clearLog()
    junction.setLayerMagnetisation("free", mag_free)
    junction.setLayerMagnetisation("bottom", mag_bottom)
    
    # set constant current 
    junction.setLayerCurrentDriver("free", ScalarDriver.getConstantDriver(currentDensity))

    HDriver = AxialDriver(
        ScalarDriver.getConstantDriver(H[0]),
        ScalarDriver.getConstantDriver(H[1]),
        ScalarDriver.getConstantDriver(H[2]))

    junction.setLayerExternalFieldDriver("all", HDriver)
    junction.runSimulation(s_time, int_step, int_step)

    mag_free = junction.getLayerMagnetisation("free")
    mag_bottom = junction.getLayerMagnetisation("bottom")
    
    log = junction.getLog()
    res = np.asarray(log["R_free_bottom"])
    N = len(res)
    y = fft(res)*(2/N)
    spectrogram[hi, :] = np.abs(y[:N//D])

freqs = fftfreq(N, d=int_step)[:N//D]

  0%|          | 0/50 [00:00<?, ?it/s]

In [141]:
import plotly.graph_objects as go
from ipywidgets import VBox

fig = go.Figure(data=go.Heatmap(x=Hscan, y=freqs/1e9, z=spectrogram[:, 1:].T/np.max(spectrogram[:, 1:])))
fig = go.FigureWidget(fig)
fig['layout'][f'xaxis1'].update(title_text='H [A/m]')
fig['layout'][f'yaxis1'].update(title_text='f [GHz]')
VBox([fig])

    'data': [{'type': 'heatmap',
              'uid': '4f7c305b-a7f8-462b-9ca3-3…